## Готовим данные для модели стратегического прогноза

In [2]:
%pylab inline
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import MySQLdb

from datetime import timedelta
import datetime

import glob

Populating the interactive namespace from numpy and matplotlib


**Загрузка данных из базы**

In [3]:
# %%time
# db_connection = MySQLdb.connect(host="194.247.15.117", 
#                      port=7806,
#                      user="bar",
#                      passwd="PASSWORD",
#                      db="Inventory")

# data = pd.read_sql('SELECT * FROM Inventory.vDynamicForecast', con=db_connection)
# db_connection.close()

# print(len(data))

11657228
Wall time: 2h 10min 5s


**Загрузка данных с локального хранилища**

In [4]:
# data.to_csv('../data/op_data.csv', sep=';')

In [64]:
data = pd.read_csv('../data/data.csv', sep=';')
data.drop('Unnamed: 0', inplace=True, axis = 1)

**Смотрим на данные**

In [5]:
data

,FlightID,Code,seatsAvail,authLevel,resHoldTotal,reportDt,orig,dest,DptDt,Carrier,fltNr,opsSuffix,fltNrDptDt,fltNrArrDt,DptTime,ArrTime
0,1,B,2,76.0,0.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
1,1,Y,4,76.0,0.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
2,1,X,10,364.0,3.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
3,1,K,1,62.0,0.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
4,1,B,2,76.0,1.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
5,1,Y,4,76.0,0.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
6,1,X,5,364.0,11.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
7,1,K,0,62.0,5.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
8,1,B,0,74.0,4.0,2016-05-28,LEDGL,MAKSA,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,1
9,1,Y,2,74.0,2.0,2016-05-28,LEDGL,MAKSA,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11657228 entries, 0 to 11657227
Data columns (total 16 columns):
FlightID        int64
Code            object
seatsAvail      int64
authLevel       float64
resHoldTotal    float64
reportDt        object
orig            object
dest            object
DptDt           object
Carrier         object
fltNr           int64
opsSuffix       object
fltNrDptDt      object
fltNrArrDt      object
DptTime         int64
ArrTime         int64
dtypes: float64(2), int64(5), object(9)
memory usage: 1.4+ GB


**Предобработка данных**

In [7]:
# номер недели
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

In [11]:
data

,FlightID,Code,seatsAvail,authLevel,resHoldTotal,reportDt,orig,dest,DptDt,Carrier,fltNr,opsSuffix,fltNrDptDt,fltNrArrDt,DptTime,ArrTime
0,1,B,2,76.0,0.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
1,1,Y,4,76.0,0.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
2,1,X,10,364.0,3.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
3,1,K,1,62.0,0.0,2016-05-28,LEDGL,MVISH,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,19
4,1,B,2,76.0,1.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
5,1,Y,4,76.0,0.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
6,1,X,5,364.0,11.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
7,1,K,0,62.0,5.0,2016-05-28,LEDGL,BOLOG,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,22
8,1,B,0,74.0,4.0,2016-05-28,LEDGL,MAKSA,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,1
9,1,Y,2,74.0,2.0,2016-05-28,LEDGL,MAKSA,2016-05-26,FPC,337,ZH,2016-05-26,2016-05-28,16,1


In [ ]:
data['flightNrSf'] = data.fltNr.astype('str') + data.opsSuffix
data.drop('fltNr', axis = 1, inplace = True)
data.drop('opsSuffix', axis = 1, inplace = True)

In [10]:
df = data.groupby(['fltNrDptDt', 'flightNrSf', 'reportDt'])['resHoldTotal'].sum()
df

KeyError: 'flightNrSf'

In [59]:
def prepareData(data):
    # drop stuff
    data.drop('seatsAvail', axis = 1, inplace = True)
    data.drop('authLevel', axis = 1, inplace = True)
    data.drop('Carrier', axis = 1, inplace = True)
    
    # encode reportDt
    
    # make flight id 
    data['flightNrSf'] = [str(data.fltNr.values[x]) + data.opsSuffix.values[x] for x in range(0, len(data.fltNr.values))]
    data.drop('fltNr', axis = 1, inplace = True)
    data.drop('opsSuffix', axis = 1, inplace = True)
    print("Made flight id")
    
    # словари для кодирования позедов, сегментов и классов вагонов
    segment_dict = dict(enumerate(set(np.append(data.orig.unique(), data.dest.unique()))))
    print('segment_dict: ' + str(segment_dict))
    flightNrSf_dict = dict(enumerate(data.flightNrSf.unique()))
    print('flightNrSf_dict: ' + str(flightNrSf_dict))
    code_dict = dict(enumerate(data.Code.unique()))
    print('code_dict: ' + str(code_dict))
    
    # кодируем сегмент
    data.orig = data.orig.map(lambda x: list(segment_dict.keys())[list(segment_dict.values()).index(x)])
    data.dest = data.dest.map(lambda x: list(segment_dict.keys())[list(segment_dict.values()).index(x)])
    
    # кодируем поезда
    data.flightNrSf = data.flightNrSf.map(lambda x: list(flightNrSf_dict.keys())[list(flightNrSf_dict.values()).index(x)])
    
    # кодируем вагоны
    data.Code = data.Code.map(lambda x: list(code_dict.keys())[list(code_dict.values()).index(x)])
    
    # make segment
    data['segment'] = data.orig.astype('str') + '_' + data.dest.astype('str')
    data.drop('orig', axis = 1, inplace = True)
    data.drop('dest', axis = 1, inplace = True)
    print("Made segment")
    
    # словарь для повторного кодирования сегментов
    segment_sec_dict = dict(enumerate(data.segment.unique()))
    print('segment_sec_dict: ' + str(segment_sec_dict))
    
    # повторно кодируем сегмент
    data.segment = data.segment.map(lambda x: list(segment_sec_dict.keys())[list(segment_sec_dict.values()).index(x)])
    print("Done with encoding")
    
    # кодируем дату
    data.DptDt = pd.DatetimeIndex(data.DptDt)
    data.fltNrDptDt = pd.DatetimeIndex(data.fltNrDptDt)
    data.fltNrArrDt = pd.DatetimeIndex(data.fltNrArrDt)
    data['DayOfWeekDpt'] = pd.DatetimeIndex(data['DptDt']).dayofweek 
    data['WeekOfMonthDpt'] = [week_of_month(x) for x in pd.DatetimeIndex(data['DptDt'])]
    data['MonthDpt'] = pd.DatetimeIndex(data['DptDt']).month 
    data['QuarterDpt'] = pd.DatetimeIndex(data['DptDt']).quarter 
    data['YearDpt'] = pd.DatetimeIndex(data['DptDt']).year
    data['IsWeekendDpt'] = data['DayOfWeekDpt'].isin([5,6])*1
    data['DayOfWeekArr'] = pd.DatetimeIndex(data['fltNrArrDt']).dayofweek 
    data['WeekOfMonthArr'] = [week_of_month(x) for x in pd.DatetimeIndex(data['DptDt'])]
    data['MonthArr'] = pd.DatetimeIndex(data['DptDt']).month 
    data['QuarterArr'] = pd.DatetimeIndex(data['DptDt']).quarter 
    data['YearArr'] = pd.DatetimeIndex(data['DptDt']).year
    data['IsWeekendArr'] = data['DayOfWeekArr'].isin([5,6])*1
    print("Dates complete")
    
    data.sort_values(['FlightID', 'DptDt'], inplace=True)
    data.reset_index(inplace=True)
    data.drop('index', axis = 1, inplace = True)
    
    # праздники
#     data['Holiday'] = 0
#     holidayUniqueCount = 0
#     holidayDays = ('2017-03-08 00:00:00', '2017-02-23 00:00:00', '2017-02-14 00:00:00', '2017-01-07 00:00:00', '2017-06-12 00:00:00', '2017-01-01 00:00:00', '2017-05-09 00:00:00',
#                    '2016-03-08 00:00:00', '2016-02-23 00:00:00', '2016-02-14 00:00:00', '2016-01-07 00:00:00', '2016-06-12 00:00:00', '2016-01-01 00:00:00', '2016-05-09 00:00:00',
#                    '2015-03-08 00:00:00', '2015-02-23 00:00:00', '2015-02-14 00:00:00', '2015-01-07 00:00:00', '2015-06-12 00:00:00', '2015-01-01 00:00:00', '2015-05-09 00:00:00',
#                    '2014-03-08 00:00:00', '2014-02-23 00:00:00', '2014-02-14 00:00:00', '2014-01-07 00:00:00', '2014-06-12 00:00:00', '2014-01-01 00:00:00', '2014-05-09 00:00:00')
    
#     for holidayIndex in holidayDays:
#         daysIncr = np.arange(-3, 4)
        
#         hs = holidayUniqueCount * 5
#         holidayCode = np.append(np.arange(1 + hs, 5 + hs), np.arange(1 + hs, 4 + hs)[::-1])
        
#         for i, dayIncr in enumerate(daysIncr):
#             holiday = pd.DatetimeIndex([holidayIndex])[0] + datetime.timedelta(days = int(dayIncr))
#             data.loc[data[data.DptDt == holiday].index.values, 'Holiday'] = holidayCode[i]
            
#         holidayUniqueCount += 1
#         if holidayUniqueCount == 7:
#             holidayUniqueCount = 0
#     print("Done with holidays")

    return data

In [73]:
%%time
data = prepareData(data)

Made flight id
segment_dict: {0: 'ZUBVP', 1: 'NOVKU', 2: 'MUROM', 3: 'SHILV', 4: 'MAKSA', 5: 'IARGL', 6: 'RZN1', 7: 'GFBMH', 8: 'NAVAS', 9: 'BEZNC', 10: 'POTMA', 11: 'VEKVK', 12: 'UZHOV', 13: 'NEREK', 14: 'SONKV', 15: 'YTNPR', 16: 'BARYS', 17: 'ARZM1', 18: 'RYBP', 19: 'SASOV', 20: 'NEKOZ', 21: 'KOVYL', 22: 'OBSHA', 23: 'KUZOV', 24: 'MOWKZ', 25: 'SYZR1', 26: 'BEZHK', 27: 'BOLOG', 28: 'OKUL', 29: 'URMAR', 30: 'SKX', 31: 'GOJ', 32: 'MVISH', 33: 'CHAPV', 34: 'SHSTH', 35: 'KOVRV', 36: 'KZNP', 37: 'KANAS', 38: 'LEDGL', 39: 'ZELDO', 40: 'VOLGA', 41: 'IVANV', 42: 'KUF', 43: 'RUZVK', 44: 'INZA'}
flightNrSf_dict: {0: '10J', 1: '50M', 2: '9J', 3: '49J', 4: '337J', 5: '337ZH'}
code_dict: {0: 'F', 1: 'C', 2: 'B', 3: 'Y', 4: 'X', 5: 'K'}
Made segment
segment_sec_dict: {0: '24_6', 1: '24_43', 2: '24_25', 3: '24_42', 4: '6_43', 5: '6_25', 6: '6_42', 7: '43_25', 8: '43_42', 9: '25_42', 10: '24_19', 11: '24_0', 12: '24_21', 13: '24_44', 14: '24_16', 15: '24_33', 16: '6_19', 17: '6_0', 18: '6_21', 19: '6

In [74]:
data.sort_values(['FlightID', 'DptDt'], inplace = True)

In [80]:
lag = datetime.timedelta(days = 365)
data['LagDptDt'] = data['fltNrDptDt'] - lag

In [76]:
lags = (1, 4, 12, 52, 3 * 52)

**Лаги по дням**

In [81]:
%%time

df = data.groupby(['fltNrDptDt', 'LagDptDt'], as_index=False)['resHoldTotal'].sum().sort_values(['fltNrDptDt'])
for lag in lags:
    df['lag_' + str(lag)] = df.LagDptDt.map(lambda x: df[(df.fltNrDptDt <= x) & (df.fltNrDptDt >= (x - timedelta(days = 7 * lag)))].resHoldTotal.mean())
    print(str(lag) + ' = lag complete')

1 = lag complete
4 = lag complete
12 = lag complete
52 = lag complete
156 = lag complete
Wall time: 1min 21s


In [84]:
data = data.merge(df.drop('LagDptDt', axis = 1).drop('resHoldTotal', axis = 1), on='fltNrDptDt')

**Лаги по поездам**

In [86]:
%%time

df = data.groupby(['fltNrDptDt', 'flightNrSf', 'LagDptDt'], as_index=False)['resHoldTotal'].sum().sort_values(['fltNrDptDt'])
for flt in df.flightNrSf.unique():
    for lag in lags:
        df.loc[df[df.flightNrSf == flt].index.values, 'lag_' + str(lag) + '_flt_' + str(flt)] = df[df.flightNrSf == flt].LagDptDt.map(lambda x: df[df.flightNrSf == flt][(df.fltNrDptDt <= x) & (df.fltNrDptDt >= (x - timedelta(days = 7 * lag)))].resHoldTotal.mean())
        print(str(lag) + ' = lag complete, ' + str(flt) + ' = flt')
    
    print(str(flt) + ' = flt complete')

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


1 = lag complete, 0 = flt
4 = lag complete, 0 = flt
12 = lag complete, 0 = flt
52 = lag complete, 0 = flt
156 = lag complete, 0 = flt
0 = flt complete
1 = lag complete, 1 = flt
4 = lag complete, 1 = flt
12 = lag complete, 1 = flt
52 = lag complete, 1 = flt
156 = lag complete, 1 = flt
1 = flt complete
1 = lag complete, 2 = flt
4 = lag complete, 2 = flt
12 = lag complete, 2 = flt
52 = lag complete, 2 = flt
156 = lag complete, 2 = flt
2 = flt complete
1 = lag complete, 4 = flt
4 = lag complete, 4 = flt
12 = lag complete, 4 = flt
52 = lag complete, 4 = flt
156 = lag complete, 4 = flt
4 = flt complete
1 = lag complete, 5 = flt
4 = lag complete, 5 = flt
12 = lag complete, 5 = flt
52 = lag complete, 5 = flt
156 = lag complete, 5 = flt
5 = flt complete
1 = lag complete, 3 = flt
4 = lag complete, 3 = flt
12 = lag complete, 3 = flt
52 = lag complete, 3 = flt
156 = lag complete, 3 = flt
3 = flt complete
Wall time: 9min 56s


In [87]:
df

,fltNrDptDt,flightNrSf,LagDptDt,resHoldTotal,lag_1_flt_0,lag_4_flt_0,lag_12_flt_0,lag_52_flt_0,lag_156_flt_0,lag_1_flt_1,...,lag_1_flt_5,lag_4_flt_5,lag_12_flt_5,lag_52_flt_5,lag_156_flt_5,lag_1_flt_3,lag_4_flt_3,lag_12_flt_3,lag_52_flt_3,lag_156_flt_3
0,2014-02-27,0,2013-02-27,303.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-02-27,1,2013-02-27,584.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-02-27,2,2013-02-27,276.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-02-28,0,2013-02-28,323.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-02-28,1,2013-02-28,629.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2014-02-28,2,2013-02-28,280.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014-03-01,0,2013-03-01,168.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014-03-01,1,2013-03-01,620.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2014-03-01,2,2013-03-01,180.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014-03-02,0,2013-03-02,188.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df[df.flightNrSf == 5]

,fltNrDptDt,flightNrSf,LagDptDt,resHoldTotal,lag_1_flt_0,lag_4_flt_0,lag_12_flt_0,lag_52_flt_0,lag_156_flt_0,lag_1_flt_1,...,lag_1_flt_5,lag_4_flt_5,lag_12_flt_5,lag_52_flt_5,lag_156_flt_5,lag_1_flt_3,lag_4_flt_3,lag_12_flt_3,lag_52_flt_3,lag_156_flt_3
1377,2015-06-01,5,2014-06-01,16.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1384,2015-06-04,5,2014-06-04,55.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1389,2015-06-06,5,2014-06-06,28.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1398,2015-06-10,5,2014-06-10,62.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407,2015-06-14,5,2014-06-14,42.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1413,2015-06-16,5,2014-06-16,56.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418,2015-06-18,5,2014-06-18,65.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1423,2015-06-20,5,2014-06-20,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1429,2015-06-22,5,2014-06-22,32.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,2015-07-06,5,2014-07-06,495.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4445 entries, 0 to 4444
Data columns (total 34 columns):
fltNrDptDt       4445 non-null datetime64[ns]
flightNrSf       4445 non-null int64
LagDptDt         4445 non-null datetime64[ns]
resHoldTotal     4445 non-null float64
lag_1_flt_0      764 non-null float64
lag_4_flt_0      764 non-null float64
lag_12_flt_0     764 non-null float64
lag_52_flt_0     764 non-null float64
lag_156_flt_0    764 non-null float64
lag_1_flt_1      537 non-null float64
lag_4_flt_1      558 non-null float64
lag_12_flt_1     614 non-null float64
lag_52_flt_1     644 non-null float64
lag_156_flt_1    644 non-null float64
lag_1_flt_2      764 non-null float64
lag_4_flt_2      764 non-null float64
lag_12_flt_2     764 non-null float64
lag_52_flt_2     764 non-null float64
lag_156_flt_2    764 non-null float64
lag_1_flt_4      142 non-null float64
lag_4_flt_4      153 non-null float64
lag_12_flt_4     153 non-null float64
lag_52_flt_4     153 non-null float64
lag

In [ ]:
data = data.merge(df.drop('LagDptDt', axis = 1).drop('resHoldTotal', axis = 1), on=['fltNrDptDt', 'flightNrSf'])

**Лаги по кодам**

In [ ]:
%%time

df = data.groupby(['fltNrDptDt', 'flightNrSf', 'Code', 'LagDptDt'], as_index=False)['resHoldTotal'].sum().sort_values(['fltNrDptDt'])
for flt in df.flightNrSf.unique():
    for code in df.Code.unique():
        for lag in lags:
            df.loc[df[df.flightNrSf == flt][df.Code == code].index.values, 'lag_' + str(lag) + '_flt_' + str(flt) + '_code_' + str(code)] = df[df.flightNrSf == flt][df.Code == code].LagDptDt.map(lambda x: df[df.flightNrSf == flt][df.Code == code][(df.fltNrDptDt <= x) & (df.fltNrDptDt >= (x - timedelta(days = 7 * lag)))].resHoldTotal.mean())
            print(str(lag) + ' = lag complete, ' + str(flt) + ' = flt, ' + str(code) + ' = code')
        
        print(str(code) + ' = code complete, ' + str(flt) + ' = flt')
    
    print(str(flt) + ' = flt complete')

In [ ]:
data = data.merge(df.drop('LagDptDt', axis = 1).drop('resHoldTotal', axis = 1), on=['fltNrDptDt', 'flightNrSf', 'Code'])

In [ ]:
df.info()

**Удаляем nan**

In [94]:
data.dropna(inplace=True)
data

,FlightID,Code,resHoldTotal,DptDt,fltNrDptDt,fltNrArrDt,DptTime,ArrTime,flightNrSf,segment,...,MonthArr,QuarterArr,YearArr,IsWeekendArr,LagDptDt,lag_1,lag_4,lag_12,lag_52,lag_156
0,25,0,0.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
1,25,1,0.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
2,25,2,5.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
3,25,3,2.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
4,25,0,2.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
5,25,1,3.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
6,25,2,1.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
7,25,3,0.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
8,25,0,0.0,2016-05-26,2016-05-26,2016-05-27,20,7,0,2,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
9,25,1,11.0,2016-05-26,2016-05-26,2016-05-27,20,7,0,2,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088


**Сохраним**

In [93]:
data

,FlightID,Code,resHoldTotal,DptDt,fltNrDptDt,fltNrArrDt,DptTime,ArrTime,flightNrSf,segment,...,MonthArr,QuarterArr,YearArr,IsWeekendArr,LagDptDt,lag_1,lag_4,lag_12,lag_52,lag_156
0,25,0,0.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
1,25,1,0.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
2,25,2,5.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
3,25,3,2.0,2016-05-26,2016-05-26,2016-05-27,20,22,0,0,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
4,25,0,2.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
5,25,1,3.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
6,25,2,1.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
7,25,3,0.0,2016-05-26,2016-05-26,2016-05-27,20,3,0,1,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
8,25,0,0.0,2016-05-26,2016-05-26,2016-05-27,20,7,0,2,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088
9,25,1,11.0,2016-05-26,2016-05-26,2016-05-27,20,7,0,2,...,5,2,2016,0,2015-05-27,1107.375,1074.344828,1042.129412,976.394521,980.712088


In [95]:
data.to_csv('../data/prepared_data.csv', sep=';')

**Общий прогзноз + год вперед**

In [97]:
wdata = pd.read_csv('../data/data.csv', sep=';')
wdata.drop('Unnamed: 0', inplace=True, axis = 1)

In [98]:
# дополним год
wdata.DptDt = pd.DatetimeIndex(wdata.DptDt)
wdata.reportDt = pd.DatetimeIndex(wdata.reportDt)
wdata.fltNrDptDt = pd.DatetimeIndex(wdata.fltNrDptDt)
wdata.fltNrArrDt = pd.DatetimeIndex(wdata.DptDt)

df = wdata[wdata['DptDt'] > wdata['DptDt'].max() - datetime.timedelta(days=365)].copy()
df['DptDt'] = df['DptDt'].map(lambda x: x + datetime.timedelta(days=365))
df['reportDt'] = df['reportDt'].map(lambda x: x + datetime.timedelta(days=365))
df['fltNrDptDt'] = df['fltNrDptDt'].map(lambda x: x + datetime.timedelta(days=365))
df['fltNrArrDt'] = df['fltNrArrDt'].map(lambda x: x + datetime.timedelta(days=365))

In [99]:
wdata = wdata.append(df)

In [100]:
wdata = prepareData(wdata)

Made flight id
segment_dict: {0: 'ZUBVP', 1: 'NOVKU', 2: 'MUROM', 3: 'SGC', 4: 'CEK', 5: 'SHILV', 6: 'MAKSA', 7: 'IARGL', 8: 'RZN1', 9: 'GFBMH', 10: 'AZOQM', 11: 'NAVAS', 12: 'LANGP', 13: 'BEZNC', 14: 'POTMA', 15: 'TJM', 16: 'VEKVK', 17: 'UZHOV', 18: 'KMNUR', 19: 'NEREK', 20: 'SONKV', 21: 'YTNPR', 22: 'WINOP', 23: 'BARYS', 24: 'ARZM1', 25: 'RYBP', 26: 'SASOV', 27: 'TORBV', 28: 'NEKOZ', 29: 'KOVYL', 30: 'OBSHA', 31: 'KUZOV', 32: 'MOWKZ', 33: 'SYZR1', 34: 'BEZHK', 35: 'BOLOG', 36: 'PYTJA', 37: 'OKUL', 38: 'KUZNT', 39: 'BAZRN', 40: 'URMAR', 41: 'SKX', 42: 'GOJ', 43: 'SVX', 44: 'MVISH', 45: 'CHAPV', 46: 'SXMBV', 47: 'SHSTH', 48: 'KOVRV', 49: 'KZNP', 50: 'PEZ1', 51: 'LEDGL', 52: 'KANAS', 53: 'ZELDO', 54: 'VOEVD', 55: 'NJC', 56: 'IVANV', 57: 'VOLGA', 58: 'KUF', 59: 'NOCKA', 60: 'RUZVK', 61: 'INZA', 62: 'TOX'}
flightNrSf_dict: {0: '10J', 1: '50M', 2: '9J', 3: '49J', 4: '337J', 5: '337ZH', 6: '138J', 7: '137YQ', 8: '102J', 9: '101J', 10: '88J', 11: '87J'}
code_dict: {0: 'F', 1: 'C', 2: 'B', 3:

ValueError: No axis named DptDt for object type <class 'pandas.core.frame.DataFrame'>

In [102]:
%%time

wdata['LagDptDt'] = wdata['fltNrDptDt'] - lag
df = wdata.groupby(['fltNrDptDt', 'LagDptDt'], as_index=False)['resHoldTotal'].sum().sort_values(['fltNrDptDt'])
for lag in lags:
    df['lag_' + str(lag)] = df.LagDptDt.map(lambda x: df[(df.fltNrDptDt <= x) & (df.fltNrDptDt >= (x - timedelta(days = 7 * lag)))].resHoldTotal.mean())
    print(str(lag) + ' = lag complete')
    
wdata = wdata.merge(df.drop('LagDptDt', axis = 1).drop('resHoldTotal', axis = 1), on='fltNrDptDt')

wdata.dropna(inplace=True)
wdata

TypeError: incompatible type for a datetime/timedelta operation [__sub__]

In [ ]:
# save 
wdata.to_csv('../data/4y_prepared_data.csv', sep=';')